### Trying to create a model for state by state data.
- First, combined the data from 2012, 2017, 2021 and 2021 to one dataframe.
- The states and year are combined to one States_year column. So, Alabama will have four rows: Alabama2012, Alabama2017, Alabama2021, Alabama2022
- The territories and DC have been excluded.
- The distribution by race has not been considered.
### Also did naïve modelling with statsmodels
- Used 10% of the 200 datasets as testing data and the rest as training data.
- Mean AP score is used as the target variable.
- Per capita income, population, density of R1/R2 universities (per million), density of public universities (per million) and density of private non-profit universities (per million) are used as features.
- Full model includes all features.
- Uni metric model includes density of R1/R2, public and private non-profit as features.
- Non-uni model includes per capita and population as features.
- Viewing non-uni model as null hypothesis, the p-value of full model is extremely low. So, null hypothesis should be excluded.
- Also, viewing uni metric model as null hypothesis, the p-value of full model is calculated to be extremely low. So, again, null hypothesis should be excluded.
- Prediction on testing data is computed. The root mean squared error is around 0.27. By comparison, majority of the mean AP scores are in 2.5-3 range.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
carnegie = pd.read_excel('data/CCIHE2021-PublicData_limited.xlsx',sheet_name='Data')
ap_participation = pd.read_excel('data/AP_data_fixed-participation.xlsx',sheet_name='AP_data_for_reference')
ap_outcome = pd.read_excel('data/AP_data_fixed-outcome.xlsx',sheet_name='Sheet1')
us_income = pd.read_excel('data/per_capita_income_federal_reserve_stlouis.xlsx')
us_population = pd.read_excel('data/population_census_bureau.xlsx')

In [3]:
ap_outcome.head(20)

,Unnamed: 0,Unnamed: 1,Total AP Exams,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,2022,2021,2017,2012
1,National,NaN,NaN,NaN,NaN,NaN
2,Overall,5,576744,447613,519961,449472
3,NaN,4,812854,711668,803224,612929
4,NaN,3,990625,926701,1073514,747418
5,NaN,2,860182,944172,1039005,690719
6,NaN,1,810450,852572,837565,659398
7,NaN,Total,4050855,3882726,4273269,3159936
8,NaN,Mean Score,2.9,2.7,2.8,2.8
9,By examinee race/ethnicity,NaN,NaN,NaN,NaN,NaN


In [4]:
index1=[1+x*10 for x in range(1,52)]
ap_participation=ap_participation.loc[index1]
index2 = [68+x*60 for x in range(51)]
ap_outcome=ap_outcome.loc[index2]
ap_outcome


,Unnamed: 0,Unnamed: 1,Total AP Exams,Unnamed: 3,Unnamed: 4,Unnamed: 5
68,NaN,Mean Score,2.5,2.4,2.3,2.3
128,NaN,Mean Score,2.9,2.8,2.9,3
188,NaN,Mean Score,2.9,2.8,2.8,2.8
248,NaN,Mean Score,2.4,2.2,2.2,2.1
308,NaN,Mean Score,3,2.8,2.8,2.9
368,NaN,Mean Score,3,2.8,2.9,2.9
428,NaN,Mean Score,3.1,3,3.2,3.3
488,NaN,Mean Score,2.7,2.5,2.7,2.6
548,NaN,Mean Score,2.4,2.3,2.1,1.9
608,NaN,Mean Score,2.7,2.6,2.6,2.5


In [5]:
ap_participation.rename( columns={'Unnamed: 0':'States', 'Percentage of HS students in Grades 10, 11, and 12 who took AP Exams':'2022','Unnamed: 2':'2021','Unnamed: 3':'2017','Unnamed: 4':'2012'}, inplace=True )

In [6]:
ap_outcome.rename( columns={'Unnamed: 0':'States', 'Total AP Exams':'2022','Unnamed: 3':'2021','Unnamed: 4':'2017','Unnamed: 5':'2012'}, inplace=True )


In [7]:
ap_outcome.head()

,States,Unnamed: 1,2022,2021,2017,2012
68,NaN,Mean Score,2.5,2.4,2.3,2.3
128,NaN,Mean Score,2.9,2.8,2.9,3
188,NaN,Mean Score,2.9,2.8,2.8,2.8
248,NaN,Mean Score,2.4,2.2,2.2,2.1
308,NaN,Mean Score,3,2.8,2.8,2.9


In [8]:
ap_outcome=ap_outcome.reset_index()
ap_participation=ap_participation.reset_index()
ap_outcome.States = ap_participation.States

In [9]:
ap_participation=ap_participation[['States','2022','2021','2017','2012']]
ap_outcome=ap_outcome[['States','2022','2021','2017','2012']]
ap_participation.head(n=10)

,States,2022,2021,2017,2012
0,Alabama,0.14,0.13,0.15,0.11
1,Alaska,0.1,0.08,0.11,0.08
2,Arizona,0.11,0.11,0.13,0.1
3,Arkansas,0.22,0.21,0.25,0.22
4,California,0.21,0.21,0.24,0.18
5,Colorado,0.2,0.19,0.22,0.18
6,Connecticut,0.23,0.23,0.23,0.18
7,Delaware,0.16,0.16,0.17,0.14
8,District Of Columbia,0.28,0.27,0.32,0.23
9,Florida,0.22,0.23,0.25,0.23


In [10]:
ap_outcome=ap_outcome.drop([8])
ap_outcome=ap_outcome.reset_index()
ap_outcome=ap_outcome[['States','2022','2021','2017','2012']]
ap_participation=ap_participation.drop([8])
ap_participation=ap_participation.reset_index()
ap_participation=ap_participation[['States','2022','2021','2017','2012']]

In [11]:
len(ap_outcome)

50

In [12]:
years = ['2022','2021','2017','2012']
state_yr=[]
score=[]
for year in years:
    for i in ap_outcome.index:
        state = ap_outcome.iloc[i]['States']+year
        outcome = ap_outcome.iloc[i][year]
        state_yr = state_yr + [state]
        score = score + [outcome]
ap_combined_outcome = {'States_year':state_yr,'AP_Score':score}

In [13]:
ap_combined_outcome = pd.DataFrame(ap_combined_outcome)
ap_combined_outcome.head()

,States_year,AP_Score
0,Alabama2022,2.5
1,Alaska2022,2.9
2,Arizona2022,2.9
3,Arkansas2022,2.4
4,California2022,3.0


In [14]:
years = ['2022','2021','2017','2012']
state_yr=[]
participation=[]
for year in years:
    for i in ap_participation.index:
        state = ap_participation.iloc[i]['States']+year
        participate = ap_participation.iloc[i][year]
        state_yr = state_yr + [state]
        participation = participation + [participate]
ap_combined_participation = {'States_year':state_yr,'AP_Participation':participation}

In [15]:
ap_combined_participation = pd.DataFrame(ap_combined_participation)

In [16]:
ap_combined_participation.head()

,States_year,AP_Participation
0,Alabama2022,0.14
1,Alaska2022,0.10
2,Arizona2022,0.11
3,Arkansas2022,0.22
4,California2022,0.21


In [17]:
sum(ap_combined_participation['States_year'] == ap_combined_participation['States_year'])

200

In [18]:
us_population.head()

,Regions,2012,2017,2021,2022
0,.Alabama,4815588,4874486,5050380,5073903
1,.Alaska,730443,739700,734923,733276
2,.Arizona,6554978,7044008,7272487,7365684
3,.Arkansas,2952164,3001345,3028443,3046404
4,.California,37948800,39358497,39145060,39040616


In [19]:
us_population['Regions']=us_population['Regions'].str.replace('.','')

In [20]:
us_population.head(n=10)

,Regions,2012,2017,2021,2022
0,Alabama,4815588,4874486,5050380,5073903
1,Alaska,730443,739700,734923,733276
2,Arizona,6554978,7044008,7272487,7365684
3,Arkansas,2952164,3001345,3028443,3046404
4,California,37948800,39358497,39145060,39040616
5,Colorado,5192647,5611885,5811596,5841039
6,Connecticut,3594547,3573297,3603691,3608706
7,Delaware,915179,956823,1004881,1019459
8,District of Columbia,634924,694906,669037,670949
9,Florida,19297822,20963613,21830708,22245521


In [21]:
us_population=us_population.drop([8])
us_population=us_population.reset_index()
us_population=us_population[['Regions',2022,2021,2017,2012]]

In [22]:
us_population.head(n=10)

,Regions,2022,2021,2017,2012
0,Alabama,5073903,5050380,4874486,4815588
1,Alaska,733276,734923,739700,730443
2,Arizona,7365684,7272487,7044008,6554978
3,Arkansas,3046404,3028443,3001345,2952164
4,California,39040616,39145060,39358497,37948800
5,Colorado,5841039,5811596,5611885,5192647
6,Connecticut,3608706,3603691,3573297,3594547
7,Delaware,1019459,1004881,956823,915179
8,Florida,22245521,21830708,20963613,19297822
9,Georgia,10913150,10790385,10410330,9901430


In [23]:
years = [2022,2021,2017,2012]
state_yr=[]
pop=[]
for year in years:
    for i in us_population.index:
        state = us_population.iloc[i]['Regions']+str(year)
        popul = us_population.iloc[i][year]
        state_yr = state_yr + [state]
        pop = pop + [popul]
us_population_combined = {'States_year':state_yr,'Population':pop}

In [24]:
us_population_combined=pd.DataFrame(us_population_combined)

In [25]:
us_income['State']=us_income['State'].str.strip()

In [26]:
us_income.head(n=10)

,State,2022,2021,2017,2012
0,Alabama,51683,50483,39975,35564
1,Alaska,69015,65563,56499,53340
2,Arizona,58968,56976,43208,36333
3,Arkansas,55323,52845,41402,36287
4,California,76941,76882,58214,47794
5,Colorado,76674,71706,54171,45490
6,Connecticut,84994,81758,69146,63555
7,Delaware,63964,59630,50002,43775
8,District of Columbia,101015,97796,79535,67470
9,Florida,64557,62242,48439,41204


In [27]:
us_income=us_income.drop([8])
us_income=us_income.reset_index()
us_income=us_income[['State',2022,2021,2017,2012]]
us_income.head(n=10)

,State,2022,2021,2017,2012
0,Alabama,51683,50483,39975,35564
1,Alaska,69015,65563,56499,53340
2,Arizona,58968,56976,43208,36333
3,Arkansas,55323,52845,41402,36287
4,California,76941,76882,58214,47794
5,Colorado,76674,71706,54171,45490
6,Connecticut,84994,81758,69146,63555
7,Delaware,63964,59630,50002,43775
8,Florida,64557,62242,48439,41204
9,Georgia,57290,56088,44838,37251


In [28]:
years = [2022,2021,2017,2012]
state_yr=[]
pci=[]
for year in years:
    for i in us_income.index:
        state = us_income.iloc[i]['State']+str(year)
        income = us_income.iloc[i][year]
        state_yr = state_yr + [state]
        pci = pci + [income]
us_income_combined = {'States_year':state_yr,'Per_capita_income':pci}

In [29]:
us_income_combined=pd.DataFrame(us_income_combined)
us_income_combined

,States_year,Per_capita_income
0,Alabama2022,51683
1,Alaska2022,69015
2,Arizona2022,58968
3,Arkansas2022,55323
4,California2022,76941
...,...,...
195,Virginia2012,49052
196,Washington2012,47057
197,West Virginia2012,35233
198,Wisconsin2012,42641


In [30]:
sum(ap_combined_outcome['States_year'] == us_income_combined['States_year'])

200

In [31]:
sum(ap_combined_outcome['States_year'] == us_population_combined['States_year'])

200

In [32]:
carnegie.head(n=10)

,unitid,name,city,stabbr,basic2000,basic2005,basic2010,basic2015,basic2018,basic2021,...,control,landgrnt,medical,hbcu,tribal,hsi,msi,womens,rooms,selindex
0,100654,Alabama A & M University,Normal,AL,16,18,18,18,18,18,...,1,1,0,1,0,0,1,0,3220,1.0
1,100663,University of Alabama at Birmingham,Birmingham,AL,15,15,15,15,15,15,...,1,0,1,0,0,0,0,0,2882,2.0
2,100690,Amridge University,Montgomery,AL,51,24,24,20,20,20,...,2,0,0,0,0,0,0,0,0,1.0
3,100706,University of Alabama in Huntsville,Huntsville,AL,16,16,15,16,16,15,...,1,0,0,0,0,0,0,0,2200,3.0
4,100724,Alabama State University,Montgomery,AL,21,18,18,19,19,17,...,1,0,0,1,0,0,1,0,2079,1.0
5,100751,The University of Alabama,Tuscaloosa,AL,15,16,16,16,15,15,...,1,0,0,0,0,0,0,0,8443,2.0
6,100760,Central Alabama Community College,Alexander City,AL,40,2,2,1,2,5,...,1,0,0,0,0,0,0,0,0,NaN
7,100812,Athens State University,Athens,AL,32,22,22,22,22,22,...,1,0,0,0,0,0,0,0,0,0.0
8,100830,Auburn University at Montgomery,Montgomery,AL,21,18,18,18,18,18,...,1,0,0,0,0,0,0,0,1200,1.0
9,100858,Auburn University,Auburn,AL,15,16,16,16,15,15,...,1,1,1,0,0,0,0,0,4823,3.0


In [33]:
print(carnegie.control.unique())
print(carnegie.basic2021.unique())

[1 2 3]
[18 15 20 17  5 22 21  4  2  7 26 24  8  1 19  3 16 32 23 13 10  9  6 33
 11 27 30 14 31 25 29 28 12]


In [34]:
territories = ['AS','GU','MP','FM','PW','VI','MH','PR','DC']  #US territories that appear in the data (including DC)

In [35]:
carnegie = carnegie.drop(carnegie[carnegie['stabbr'].isin(territories)].index)  #Drop all the states from the list "territories"

In [36]:
len(carnegie.stabbr.unique())

50

In [37]:
relevant_columns = ['name','city','stabbr','basic2010','basic2015','basic2021','control']

In [38]:
carnegie = carnegie[relevant_columns]

In [39]:
state_abbreviation = pd.read_csv('data/State Abbreviation.csv')
state_abbreviation[state_abbreviation['stabbr'] == 'AL'].State

1    Alabama
Name: State, dtype: object

In [40]:
carnegie

,name,city,stabbr,basic2010,basic2015,basic2021,control
0,Alabama A & M University,Normal,AL,18,18,18,1
1,University of Alabama at Birmingham,Birmingham,AL,15,15,15,1
2,Amridge University,Montgomery,AL,24,20,20,2
3,University of Alabama in Huntsville,Huntsville,AL,15,16,15,1
4,Alabama State University,Montgomery,AL,18,19,17,1
...,...,...,...,...,...,...,...
3933,California University of Science and Medicine,Colton,CA,-2,-2,25,2
3934,California Institute of Arts & Technology-Nati...,National City,CA,-2,-2,11,3
3936,The Judge Advocate General's School,Charlottesville,VA,-2,-2,32,1
3937,United States Army War College,Carlisle,PA,-2,-2,32,1


In [41]:
def stabbr_to_state(state_abbrev):
    state = state_abbreviation[state_abbreviation['stabbr'] == state_abbrev].State.values[0]
    return state

In [42]:
carnegie['States']=carnegie['stabbr'].apply(stabbr_to_state)

In [43]:
carnegie

,name,city,stabbr,basic2010,basic2015,basic2021,control,States
0,Alabama A & M University,Normal,AL,18,18,18,1,Alabama
1,University of Alabama at Birmingham,Birmingham,AL,15,15,15,1,Alabama
2,Amridge University,Montgomery,AL,24,20,20,2,Alabama
3,University of Alabama in Huntsville,Huntsville,AL,15,16,15,1,Alabama
4,Alabama State University,Montgomery,AL,18,19,17,1,Alabama
...,...,...,...,...,...,...,...,...
3933,California University of Science and Medicine,Colton,CA,-2,-2,25,2,California
3934,California Institute of Arts & Technology-Nati...,National City,CA,-2,-2,11,3,California
3936,The Judge Advocate General's School,Charlottesville,VA,-2,-2,32,1,Virginia
3937,United States Army War College,Carlisle,PA,-2,-2,32,1,Pennsylvania


In [44]:
R1R2_2021 = {'States':[],'R1count':[],'R2count':[]}
for i in carnegie.States.unique():
    R1R2_2021['States'].append(i)
    R1R2_2021['R1count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2021.eq(15)]))
    R1R2_2021['R2count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2021.eq(16)]))

R1R2_2021=pd.DataFrame(R1R2_2021)
R1R2_2021['R1plusR2']=R1R2_2021.R1count+R1R2_2021.R2count
R1R2_2021.head(n=10)

,States,R1count,R2count,R1plusR2
0,Alabama,4,1,5
1,Alaska,0,1,1
2,Washington,2,0,2
3,Arizona,2,2,4
4,New Mexico,1,1,2
5,Arkansas,1,2,3
6,California,11,14,25
7,Minnesota,1,0,1
8,Colorado,5,1,6
9,Connecticut,2,0,2


In [45]:
R1R2_2015 = {'States':[],'R1count':[],'R2count':[]}
for i in carnegie.States.unique():
    R1R2_2015['States'].append(i)
    R1R2_2015['R1count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2015.eq(15)]))
    R1R2_2015['R2count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2015.eq(16)]))

R1R2_2015=pd.DataFrame(R1R2_2015)
R1R2_2015['R1plusR2']=R1R2_2015.R1count+R1R2_2015.R2count
R1R2_2015.head(n=10)

,States,R1count,R2count,R1plusR2
0,Alabama,1,4,5
1,Alaska,0,1,1
2,Washington,2,0,2
3,Arizona,2,1,3
4,New Mexico,1,1,2
5,Arkansas,1,0,1
6,California,11,4,15
7,Minnesota,1,0,1
8,Colorado,2,4,6
9,Connecticut,2,0,2


In [46]:
R1R2_2010 = {'States':[],'R1count':[],'R2count':[]}
for i in carnegie.States.unique():
    R1R2_2010['States'].append(i)
    R1R2_2010['R1count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2010.eq(15)]))
    R1R2_2010['R2count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2010.eq(16)]))

R1R2_2010=pd.DataFrame(R1R2_2010)
R1R2_2010['R1plusR2']=R1R2_2010.R1count+R1R2_2010.R2count
len(R1R2_2010.index)

50

In [47]:
public_private = {'States':[],'public':[],'private_not_profit':[],'private_for_profit':[]}
for i in carnegie.States.unique():
    public_private['States'].append(i)
    public_private['public'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.control.eq(1)]))
    public_private['private_not_profit'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.control.eq(2)]))
    public_private['private_for_profit'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.control.eq(3)]))

public_private = pd.DataFrame(public_private)
public_private

,States,public,private_not_profit,private_for_profit
0,Alabama,38,18,4
1,Alaska,4,3,1
2,Washington,42,23,7
3,Arizona,26,13,27
4,New Mexico,28,3,5
5,Arkansas,34,17,2
6,California,153,151,121
7,Minnesota,45,33,7
8,Colorado,29,9,24
9,Connecticut,19,17,2


In [48]:
public_private[public_private['States'] == 'New York']

,States,public,private_not_profit,private_for_profit
32,New York,81,185,29


In [49]:
R1R2_2021.iloc[5].R1plusR2

3

In [50]:
state_yr=[]
R1R2_uni_count=[]
R1_uni_count = []

for i in R1R2_2021.index:
    state = R1R2_2021.iloc[i]['States']+str(2022)
    r1r2count = R1R2_2021.iloc[i]['R1plusR2']
    r1count = R1R2_2021.iloc[i]['R1count']
    state_yr = state_yr + [state]
    R1R2_uni_count = R1R2_uni_count + [r1r2count]
    R1_uni_count = R1_uni_count + [r1count]

for i in R1R2_2021.index:
    state = R1R2_2021.iloc[i]['States']+str(2021)
    r1r2count = R1R2_2021.iloc[i]['R1plusR2']
    r1count = R1R2_2021.iloc[i]['R1count']
    state_yr = state_yr + [state]
    R1R2_uni_count = R1R2_uni_count + [r1r2count]
    R1_uni_count = R1_uni_count + [r1count]

for i in R1R2_2021.index:
    state = R1R2_2021.iloc[i]['States']+str(2017)
    r1r2count = R1R2_2021.iloc[i]['R1plusR2']
    r1count = R1R2_2021.iloc[i]['R1count']
    state_yr = state_yr + [state]
    R1R2_uni_count = R1R2_uni_count + [r1r2count]
    R1_uni_count = R1_uni_count + [r1count]

for i in R1R2_2021.index:
    state = R1R2_2021.iloc[i]['States']+str(2012)
    r1r2count = R1R2_2021.iloc[i]['R1plusR2']
    r1count = R1R2_2021.iloc[i]['R1count']
    state_yr = state_yr + [state]
    R1R2_uni_count = R1R2_uni_count + [r1r2count]
    R1_uni_count = R1_uni_count + [r1count]

state_yr

['Alabama2022',
 'Alaska2022',
 'Washington2022',
 'Arizona2022',
 'New Mexico2022',
 'Arkansas2022',
 'California2022',
 'Minnesota2022',
 'Colorado2022',
 'Connecticut2022',
 'Delaware2022',
 'Florida2022',
 'Georgia2022',
 'Hawaii2022',
 'Idaho2022',
 'Illinois2022',
 'Indiana2022',
 'Iowa2022',
 'Kansas2022',
 'Missouri2022',
 'Kentucky2022',
 'Louisiana2022',
 'Maine2022',
 'Maryland2022',
 'Massachusetts2022',
 'Michigan2022',
 'Mississippi2022',
 'Montana2022',
 'Nebraska2022',
 'Nevada2022',
 'New Hampshire2022',
 'New Jersey2022',
 'New York2022',
 'North Carolina2022',
 'North Dakota2022',
 'Ohio2022',
 'Oklahoma2022',
 'Oregon2022',
 'Pennsylvania2022',
 'Rhode Island2022',
 'South Carolina2022',
 'South Dakota2022',
 'Tennessee2022',
 'Texas2022',
 'Utah2022',
 'Vermont2022',
 'Virginia2022',
 'West Virginia2022',
 'Wisconsin2022',
 'Wyoming2022',
 'Alabama2021',
 'Alaska2021',
 'Washington2021',
 'Arizona2021',
 'New Mexico2021',
 'Arkansas2021',
 'California2021',
 'Minne

In [51]:
years = [2022,2021,2017,2012]
public_uni_count = []
private_notprofit_count = []
private_forprofit_count = []
for year in years:
    for i in public_private.index:
        state = public_private.iloc[i]['States']+str(year)
        public = public_private.iloc[i]['public']
        private_np = public_private.iloc[i]['private_not_profit']
        private_fp = public_private.iloc[i]['private_for_profit']
        public_uni_count = public_uni_count + [public]
        private_notprofit_count = private_notprofit_count + [private_np]
        private_forprofit_count = private_forprofit_count + [private_fp]

len(public_uni_count)

200

In [52]:
uni_classification = {'States_year':state_yr,'R1_uni_count':R1_uni_count,'R1R2_uni_count':R1R2_uni_count, 'public_uni_count':public_uni_count,'private_notprofit_count':private_notprofit_count,'private_forprofit_count':private_forprofit_count}

In [53]:
uni_classification = pd.DataFrame(uni_classification)
uni_classification

,States_year,R1_uni_count,R1R2_uni_count,public_uni_count,private_notprofit_count,private_forprofit_count
0,Alabama2022,4,5,38,18,4
1,Alaska2022,0,1,4,3,1
2,Washington2022,2,2,42,23,7
3,Arizona2022,2,4,26,13,27
4,New Mexico2022,1,2,28,3,5
...,...,...,...,...,...,...
195,Vermont2012,0,1,5,11,0
196,Virginia2012,5,7,42,40,26
197,West Virginia2012,1,2,22,9,10
198,Wisconsin2012,2,3,33,30,4


In [54]:
ap_combined_outcome = ap_combined_outcome.sort_values('States_year').reset_index()[['States_year','AP_Score']]
us_income_combined = us_income_combined.sort_values('States_year').reset_index()[['States_year','Per_capita_income']]
us_population_combined = us_population_combined.sort_values('States_year').reset_index()[['States_year','Population']]
uni_classification = uni_classification.sort_values('States_year').reset_index()[['States_year','R1R2_uni_count','public_uni_count','private_notprofit_count']]

In [55]:
sum(uni_classification['States_year']==ap_combined_outcome['States_year'])

200

In [56]:
ap_overall_statelevel = uni_classification
ap_overall_statelevel['mean_AP_Score'] = ap_combined_outcome['AP_Score']
ap_overall_statelevel['Per_capita_income'] = us_income_combined['Per_capita_income']
ap_overall_statelevel['Population'] = us_population_combined['Population']
ap_overall_statelevel.to_excel('data/combined_data_2012-17-21-22_statebystate_overall.xlsx')

In [57]:
ap_overall_statelevel['R1R2_per_million'] = ap_overall_statelevel['R1R2_uni_count']*1000000/ap_overall_statelevel['Population']  #Count of R1/R2 uni per million population
ap_overall_statelevel['public_per_million'] = ap_overall_statelevel['public_uni_count']*1000000/ap_overall_statelevel['Population']  #Count of public uni per million population
ap_overall_statelevel['private_notprofit_per_million'] = ap_overall_statelevel['private_notprofit_count']*1000000/ap_overall_statelevel['Population']  #Count of private not profit uni per million population


In [58]:
ap_overall_statelevel

,States_year,R1R2_uni_count,public_uni_count,private_notprofit_count,mean_AP_Score,Per_capita_income,Population,R1R2_per_million,public_per_million,private_notprofit_per_million
0,Alabama2012,5,38,18,2.3,35564,4815588,1.038295,7.891041,3.737861
1,Alabama2017,5,38,18,2.3,39975,4874486,1.025749,7.795694,3.692697
2,Alabama2021,5,38,18,2.4,50483,5050380,0.990025,7.524186,3.564088
3,Alabama2022,5,38,18,2.5,51683,5073903,0.985435,7.489304,3.547565
4,Alaska2012,1,4,3,3.0,53340,730443,1.369032,5.476129,4.107097
...,...,...,...,...,...,...,...,...,...,...
195,Wisconsin2022,3,33,30,3.1,61992,5890543,0.509291,5.602200,5.092909
196,Wyoming2012,1,8,0,2.8,53596,576305,1.735192,13.881538,0.000000
197,Wyoming2017,1,8,0,2.7,56377,578931,1.727322,13.818573,0.000000
198,Wyoming2021,1,8,0,2.6,70973,579548,1.725483,13.803861,0.000000


In [59]:
features = ['Per_capita_income','R1R2_per_million','public_per_million','private_notprofit_per_million','Population']

In [60]:
from sklearn.model_selection import train_test_split
training, testing = train_test_split(ap_overall_statelevel, test_size = 0.1, random_state = 216)

In [61]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [62]:
full_model = smf.ols('mean_AP_Score ~ Per_capita_income + R1R2_per_million + public_per_million + private_notprofit_per_million + Population', data=training).fit()
pci_model = smf.ols('mean_AP_Score ~ Per_capita_income', data =training).fit()
r1r2_model = smf.ols('mean_AP_Score ~ R1R2_per_million', data = training).fit()
unimetric_model = smf.ols('mean_AP_Score ~ R1R2_per_million + public_per_million + private_notprofit_per_million', data = training).fit()
nonuni_model = smf.ols('mean_AP_Score ~ Per_capita_income + Population', data=training).fit()

In [63]:
full_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          mean_AP_Score   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     10.05
Date:                Wed, 06 Nov 2024   Prob (F-statistic):           1.82e-08
Time:                        03:27:45   Log-Likelihood:                 7.7578
No. Observations:                 180   AIC:                            -3.516
Df Residuals:                     174   BIC:                             15.64
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         2.2970      0.098     23.423      0.000       2.103       2.491
Per_capita_income              7.095e-06   1.54e-06      4.601      0.000    4.05e-06    1.01e-05
R1R2_per_million                  0.0239      0.044      0.542      0.588      -0.063       0.111
public_per_million               -0.0043      0.006     -0.780      0.436      -0.015       0.007
private_notprofit_per_million     0.0271      0.006      4.536      0.000       0.015       0.039
Population                     1.414e-10    2.7e-09      0.052      0.958   -5.19e-09    5.47e-09
==============================================================================
Omnibus:                        3.190   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.203   Jarque-Bera (JB):                3.251
Skew:                          -0.310   Prob(JB):                        0.197
Kurtosis:                       2.779   Cond. No.                     5.50e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.5e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
f_test_nonuni = full_model.compare_f_test(nonuni_model)
f_test_unimetric = full_model.compare_f_test(unimetric_model)
print("p-value of full compared to non-university metric model:", f_test_nonuni[1])
print("p-value of full compared to university metric model:", f_test_unimetric[1])


p-value of full compared to non-university metric model: 5.020910331355542e-05
p-value of full compared to university metric model: 3.544711626939914e-05


### The p-values are extremely small. So, it seems both university and non-university metrics are essential for good modeling. So, we choose full model as our final model and use it on the testing data.

In [65]:
predictions = full_model.predict(testing)  # Compute the predictions on the testing data

In [66]:
from statsmodels.tools.eval_measures import rmse

rmse = rmse(testing['mean_AP_Score'], predictions)
print('root mean squared error is: ',rmse)

root mean squared error is:  0.2693392353363362


In [67]:
np.mean(testing['mean_AP_Score'])

2.67

In [68]:
np.mean(predictions)

2.790985730695579